In [11]:
import os
import numpy as np
from unidecode import unidecode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from PIL import Image
import shutil

data_dir = "./korean_food"

# 1. 폴더명 한글에서 영어 변환
for folder in os.listdir(data_dir):
    src = os.path.join(data_dir, folder)
    if os.path.isdir(src):
        eng_name = unidecode(folder).lower().replace(" ", "_")
        dst = os.path.join(data_dir, eng_name)
        if src != dst and not os.path.exists(dst):
            os.rename(src, dst)
            print(f"{folder} -> {eng_name} 변경 완료")
        elif src != dst:
            print(f"{dst} 이미 존재")

In [ ]:
# 2. 이미지 파일 검증 및 정리
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
removed_count = 0

def is_valid_image(file_path):
    """이미지 파일이 유효한지 철저히 검사"""
    try:
        # 1. 파일 크기 확인 (0바이트 파일 제거)
        if os.path.getsize(file_path) == 0:
            print(f"0바이트 파일: {file_path}")
            return False
            
        # 2. PIL로 이미지 열기 및 검증
        with Image.open(file_path) as img:
            img.verify()  # 이미지 무결성 검증
            
        # 3. 이미지를 다시 열어서 실제 로드 테스트
        with Image.open(file_path) as img:
            img.load()  # 실제 이미지 데이터 로드
            
            # 4. 최소 크기 확인 (너무 작은 이미지 제거)
            if img.size[0] < 32 or img.size[1] < 32:
                print(f"너무 작은 이미지: {file_path} - {img.size}")
                return False
                
            # 5. RGB 변환 테스트
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
        # 6. TensorFlow로 로딩 테스트
        tf_img = tf.keras.utils.load_img(file_path, target_size=(224, 224))
        tf_array = tf.keras.utils.img_to_array(tf_img)
        
        return True
        
    except Exception as e:
        print(f"손상된 이미지: {file_path} - {str(e)}")
        return False

# 모든 파일 검증 및 정리
for root, dirs, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        
        # 이미지 확장자가 아닌 파일 제거
        if not file.lower().endswith(valid_extensions):
            try:
                os.remove(file_path)
                print(f"제거 (비이미지): {file}")
                removed_count += 1
            except Exception as e:
                print(f"제거 실패: {file_path} - {str(e)}")
        
        # 이미지 파일 검증
        elif not is_valid_image(file_path):
            try:
                os.remove(file_path)
                print(f"제거 (손상됨): {file}")
                removed_count += 1
            except Exception as e:
                print(f"제거 실패: {file_path} - {str(e)}")

print(f"총 {removed_count}개 파일 제거됨")

# 추가: 숨김 파일 및 시스템 파일 제거
system_files = ['.DS_Store', 'Thumbs.db', '._.DS_Store', 'desktop.ini']
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file in system_files or file.startswith('._'):
            file_path = os.path.join(root, file)
            try:
                os.remove(file_path)
                print(f"시스템 파일 제거: {file}")
                removed_count += 1
            except Exception:
                pass

C:\Users\anwlro0420\AppData\Roaming\Python\Python312\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [3]:
# 3. 각 클래스별 이미지 개수 확인
class_counts = {}
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        count = len([f for f in os.listdir(folder_path) 
                    if f.lower().endswith(valid_extensions)])
        class_counts[folder] = count
        print(f"  {folder}: {count}개")

  aehobagboggeum: 1000개
  albab: 1000개
  baecugimci: 1000개
  baeggimci: 998개
  bibimbab: 996개
  bibimnaengmyeon: 1001개
  bossam: 1001개
  bucugimci: 997개
  bugeosgug: 1000개
  bulgogi: 1000개
  conggaggimci: 998개
  cueotang: 1003개
  dalgboggeumtang: 1002개
  dalggalbi: 1000개
  dalggyejang: 1003개
  ddangkongjorim: 1001개
  ddeogboggi: 996개
  ddeoggalbi: 1000개
  ddeogggoci: 1095개
  ddeoggug_mandugug: 997개
  deodeoggui: 1000개
  doenjangjjigae: 999개
  donggeurangddaeng: 1000개
  dongtaejjigae: 1000개
  dorajimucim: 1000개
  dotorimug: 1002개
  dubugimci: 1000개
  dubujorim: 1000개
  eomugboggeum: 1001개
  gajiboggeum: 1000개
  galbigui: 1000개
  galbijjim: 1000개
  galbitang: 1000개
  galcigui: 1000개
  galcijorim: 1005개
  gamjacaeboggeum: 1000개
  gamjajeon: 1001개
  gamjajorim: 1001개
  gamjatang: 1003개
  ganjanggejang: 1001개
  gasgimci: 995개
  geonsaeuboggeum: 1000개
  ggaesipjangajji: 1001개
  ggagdugi: 1000개
  ggomagjjim: 1004개
  ggongcijorim: 999개
  ggulddeog: 1001개
  ggwarigocumucim: 998개
  gimbab: 999개


In [4]:
#이미지 크기 분석
image_sizes = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(valid_extensions):
            try:
                with Image.open(os.path.join(root, file)) as img:
                    image_sizes.append(img.size)
            except Exception:
                continue

if image_sizes:
    widths = [size[0] for size in image_sizes]
    heights = [size[1] for size in image_sizes]
    print(f"이미지 개수: {len(image_sizes)}개")
    print(f"너비 범위: {min(widths)} ~ {max(widths)} (평균: {sum(widths)//len(widths)})")
    print(f"높이 범위: {min(heights)} ~ {max(heights)} (평균: {sum(heights)//len(heights)})")
    print(f"가장 큰 이미지: {max(widths)}×{max(heights)}")


📏 이미지 크기 분석 중...
   이미지 개수: 150507개
   너비 범위: 121 ~ 6048 (평균: 601)
   높이 범위: 91 ~ 4208 (평균: 453)
   가장 큰 이미지: 6048×4208


In [9]:
# 4. 데이터셋 불러오기
img_size = (224, 224)
batch_size = 64
seed = 123

try:
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=seed,
        image_size=img_size,  # 224x224로 기본 리사이징
        batch_size=batch_size,
        label_mode='int',
        crop_to_aspect_ratio=False  # 크롭하지 않고 원본 유지
    )
    
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=seed,
        image_size=img_size,
        batch_size=batch_size,
        label_mode='int',
        crop_to_aspect_ratio=False  # 크롭하지 않고 원본 유지
    )
    
    # 추가 전처리 함수 (최적화된 버전)
    def smart_resize_with_padding_fast(image, label):
        """빠른 종횡비 유지 리사이징"""
        # 이미지를 float32로 변환
        image = tf.cast(image, tf.float32)
        
        # 간단한 resize_with_pad 사용 (더 빠름)
        image = tf.image.resize_with_pad(
            image, 
            target_height=img_size[0],
            target_width=img_size[1],
            method='bilinear'
        )
        
        # 크기 고정
        image = tf.ensure_shape(image, img_size + (3,))
        
        return tf.cast(image, tf.uint8), label
    
    # 빠른 리사이징 적용
    train_ds = train_ds.map(smart_resize_with_padding_fast, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.map(smart_resize_with_padding_fast, num_parallel_calls=tf.data.AUTOTUNE)
    
    class_names = train_ds.class_names
    print(f"데이터셋 로딩 완료")
    print(f"클래스 수: {len(class_names)}")
    print(f"클래스명: {class_names}")
    
except Exception as e:
    print(f"데이터셋 로딩 실패: {str(e)}")
    raise

Found 150507 files belonging to 150 classes.
Using 120406 files for training.
Found 150507 files belonging to 150 classes.
Using 30101 files for validation.
✅ 데이터셋 로딩 완료
   클래스 수: 150
   클래스명: ['aehobagboggeum', 'albab', 'baecugimci', 'baeggimci', 'bibimbab', 'bibimnaengmyeon', 'bossam', 'bucugimci', 'bugeosgug', 'bulgogi', 'conggaggimci', 'cueotang', 'dalgboggeumtang', 'dalggalbi', 'dalggyejang', 'ddangkongjorim', 'ddeogboggi', 'ddeoggalbi', 'ddeogggoci', 'ddeoggug_mandugug', 'deodeoggui', 'doenjangjjigae', 'donggeurangddaeng', 'dongtaejjigae', 'dorajimucim', 'dotorimug', 'dubugimci', 'dubujorim', 'eomugboggeum', 'gajiboggeum', 'galbigui', 'galbijjim', 'galbitang', 'galcigui', 'galcijorim', 'gamjacaeboggeum', 'gamjajeon', 'gamjajorim', 'gamjatang', 'ganjanggejang', 'gasgimci', 'geonsaeuboggeum', 'ggaesipjangajji', 'ggagdugi', 'ggomagjjim', 'ggongcijorim', 'ggulddeog', 'ggwarigocumucim', 'gimbab', 'gimciboggeumbab', 'gimcijeon', 'gimcijjigae', 'gimcijjim', 'gobcanggui', 'gobcangjeongol

In [10]:
# 5. 데이터 전처리 및 성능 최적화
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache()  # 메모리에 캐싱 (첫 epoch 후 빨라짐)
train_ds = train_ds.shuffle(1000, reseed_each_iteration=True)  # 셔플
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)  # 백그라운드 로딩

val_ds = val_ds.cache()  # 검증 데이터도 캐싱
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# 6. 모델 정의
normalization_layer = layers.Rescaling(1./255)

base_model = keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # 처음에는 feature extractor로만 사용

model = keras.Sequential([
    normalization_layer,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),  # 드롭아웃 비율 증가
    layers.Dense(128, activation="relu"),  # 중간 레이어 추가
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation="softmax")
])

# 7. 모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # 학습률 명시
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# 8. 콜백 설정
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7
    )
]

# 9. 모델 학습
epochs = 20

try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    print("학습 완료")
    
except Exception as e:
    print(f"학습 중 오류 발생: {str(e)}")
    raise

# 10. 모델 평가
test_loss, test_accuracy = model.evaluate(val_ds, verbose=0)
print(f"검증 정확도: {test_accuracy:.4f}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/20
 824/3763 ━━━━━━━━━━━━━━━━━━━━ 23:01 470ms/step - accuracy: 0.0071 - loss: 5.0182학습 중 오류 발생: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_17910]


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_17910]

In [ ]:
# 11. 예측 함수
# 자동 역매핑: 영어 폴더명 → 한글 폴더명
original_folders = {}
for item in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, item)):
        # 현재 폴더명이 변환된 것이라면 원래 이름을 찾기 어려우므로
        # 변환된 이름을 그대로 사용하거나, 별도 매핑 파일 필요
        original_folders[item] = item

def predict_image(model, img_path, class_names):
    """이미지를 예측하는 함수"""
    try:
        # 이미지 로드 및 전처리
        img = tf.keras.utils.load_img(img_path, target_size=img_size)
        x = tf.keras.utils.img_to_array(img)
        x = tf.expand_dims(x, axis=0)  # 배치 차원 추가
        
        # 예측
        predictions = model.predict(x, verbose=0)
        predicted_class_idx = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_idx]
        predicted_class = class_names[predicted_class_idx]
        
        return predicted_class, confidence
        
    except Exception as e:
        print(f"예측 중 오류: {str(e)}")
        return None, 0.0

In [ ]:
# 12. 예측 테스트 (이미지 파일이 있는 경우)
test_img_path = "Img_051_0971.jpg"
if os.path.exists(test_img_path):
    predicted_food, confidence = predict_image(model, test_img_path, class_names)
    if predicted_food:
        print(f"\n🍽️ 예측 결과:")
        print(f"   음식명: {predicted_food}")
        print(f"   신뢰도: {confidence:.4f}")
    else:
        print(f"\n예측 실패: {test_img_path}")
else:
    print(f"\n테스트 이미지 파일을 찾을 수 없습니다: {test_img_path}")

# 13. 모델 저장
model_save_path = "korean_food_classifier.keras"
model.save(model_save_path)
print(f"\n모델이 저장되었습니다: {model_save_path}")

print("\n모든 작업이 완료되었습니다!")